# Stop Sign Detection using Sliding Window Algorithm

This notebook implements a sliding window algorithm to detect stop signs in images. The algorithm:
1. Slides a window of fixed size across the image at multiple scales
2. Extracts HOG features from each window
3. Classifies each window using a trained SVM model
4. Applies non-maximum suppression to remove overlapping detections
5. Draws bounding boxes around detected stop signs


In [4]:
import os
import cv2
import numpy as np
from sklearn import svm
from skimage.feature import hog
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle


## Setup: Load Data and Train Model (if not already trained)


In [5]:
# Define image size for training
size = (128, 128)

# Dynamic paths that work regardless of where the project is located
# Get project root directory - works from any location
current_dir = os.path.abspath(os.getcwd())
project_root = current_dir

# Look for project root by finding the Data folder
# Go up directories until we find Data folder or reach filesystem root
max_levels = 5  # Prevent infinite loops
for _ in range(max_levels):
    if os.path.exists(os.path.join(project_root, "Data")) and os.path.exists(os.path.join(project_root, "Data2")):
        break
    parent = os.path.dirname(project_root)
    if parent == project_root:  # Reached filesystem root
        break
    project_root = parent

# Build paths relative to project root
path_stop = os.path.join(project_root, "Data")  # Stop signs folder
path_non_stop = os.path.join(project_root, "Data2")  # Non-stop signs folder

# Print paths for debugging
print(f"Project root: {project_root}")
print(f"Stop signs path: {path_stop}")
print(f"Non-stop signs path: {path_non_stop}")

# Lists to store images and labels
images = []
labels = []

# Process stop sign images (label 1)
if not os.path.exists(path_stop):
    print(f"Error: Stop signs folder not found at: {path_stop}")
else:
    print(f"Loading stop sign images from: {path_stop}")
    for file_name in os.listdir(path_stop):
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img_path = os.path.join(path_stop, file_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, size)  # Resize to defined size
                images.append(img)
                labels.append(1)  # Label for stop signs
    print(f"Loaded {sum(labels)} stop sign images")

# Process non-stop sign images (label 0)
if not os.path.exists(path_non_stop):
    print(f"Error: Non-stop signs folder not found at: {path_non_stop}")
else:
    print(f"Loading non-stop sign images from: {path_non_stop}")
    non_stop_count = 0
    for file_name in os.listdir(path_non_stop):
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img_path = os.path.join(path_non_stop, file_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, size)  # Resize to defined size
                images.append(img)
                labels.append(0)  # Label for non-stop signs
                non_stop_count += 1
    print(f"Loaded {non_stop_count} non-stop sign images")

print(f"\nTotal images loaded: {len(images)}")
print(f"Stop signs: {sum(labels)}, Non-stop signs: {len(labels) - sum(labels)}")


Project root: C:\Users\ENVY\Desktop\CSE445\CSE445.5-MSRb-Group-8
Stop signs path: C:\Users\ENVY\Desktop\CSE445\CSE445.5-MSRb-Group-8\Data
Non-stop signs path: C:\Users\ENVY\Desktop\CSE445\CSE445.5-MSRb-Group-8\Data2
Loading stop sign images from: C:\Users\ENVY\Desktop\CSE445\CSE445.5-MSRb-Group-8\Data
Loaded 50 stop sign images
Loading non-stop sign images from: C:\Users\ENVY\Desktop\CSE445\CSE445.5-MSRb-Group-8\Data2
Loaded 35 non-stop sign images

Total images loaded: 85
Stop signs: 50, Non-stop signs: 35


In [6]:
# Extract HOG features from all images
print("Extracting HOG features...")
hog_features = []
for image in images:
    features, _ = hog(image, pixels_per_cell=(9, 9), cells_per_block=(2, 2), 
                      orientations=10, block_norm='L2-Hys', visualize=False)
    hog_features.append(features)

X = np.array(hog_features)
y = np.array(labels)

print(f"HOG features shape: {X.shape}")

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


Extracting HOG features...


ValueError: too many values to unpack (expected 2)

In [ ]:
# Train SVM model (or load if already trained)
model_path = os.path.join(project_root, "Support", "sliding_window_model.pkl")

if os.path.exists(model_path):
    print("Loading pre-trained model...")
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    print("Model loaded successfully!")
else:
    print("Training SVM model...")
    model = svm.SVC(kernel='linear', C=1.0, probability=True)  # probability=True for confidence scores
    model.fit(X_train, y_train)
    
    # Save the model
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    print("Model trained and saved!")
    
    # Evaluate on test set
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model accuracy on test set: {accuracy:.4f}")


## Sliding Window Detection Functions


In [ ]:
def sliding_window(image, window_size=(128, 128), step_size=32):
    """
    Slide a window across the image.
    
    Args:
        image: Input image (grayscale)
        window_size: Size of the sliding window (width, height)
        step_size: Step size for sliding the window
    
    Yields:
        (x, y, window): Coordinates and the window image
    """
    for y in range(0, image.shape[0] - window_size[1] + 1, step_size):
        for x in range(0, image.shape[1] - window_size[0] + 1, step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])


def detect_stop_signs(image, model, window_size=(128, 128), step_size=32, 
                      scales=[1.0, 0.75, 0.5, 1.25, 1.5], threshold=0.5):
    """
    Detect stop signs in an image using sliding window approach.
    
    Args:
        image: Input image (grayscale or color)
        model: Trained classifier model
        window_size: Base size of the sliding window
        step_size: Step size for sliding
        scales: List of scales to search at
        threshold: Confidence threshold for detections
    
    Returns:
        detections: List of (x, y, w, h, confidence) tuples
    """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    detections = []
    
    # Search at multiple scales
    for scale in scales:
        # Resize image
        scaled_width = int(gray.shape[1] * scale)
        scaled_height = int(gray.shape[0] * scale)
        scaled_image = cv2.resize(gray, (scaled_width, scaled_height))
        
        # Adjust window size for this scale
        scaled_window = (int(window_size[0] * scale), int(window_size[1] * scale))
        scaled_step = int(step_size * scale)
        
        # Slide window across scaled image
        for x, y, window in sliding_window(scaled_image, scaled_window, scaled_step):
            # Resize window to model input size if needed
            if window.shape != window_size[::-1]:  # window_size is (width, height), shape is (height, width)
                window = cv2.resize(window, window_size)
            
            # Extract HOG features
            try:
                features, _ = hog(window, pixels_per_cell=(9, 9), cells_per_block=(2, 2), 
                                 orientations=10, block_norm='L2-Hys', visualize=False)
                features = features.reshape(1, -1)
                
                # Predict
                prediction = model.predict(features)[0]
                confidence = model.predict_proba(features)[0][1]  # Probability of being stop sign
                
                # If detected as stop sign with sufficient confidence
                if prediction == 1 and confidence >= threshold:
                    # Scale coordinates back to original image size
                    orig_x = int(x / scale)
                    orig_y = int(y / scale)
                    orig_w = int(scaled_window[0] / scale)
                    orig_h = int(scaled_window[1] / scale)
                    
                    detections.append((orig_x, orig_y, orig_w, orig_h, confidence))
            except:
                continue
    
    return detections


def non_max_suppression(detections, overlap_threshold=0.3):
    """
    Apply non-maximum suppression to remove overlapping detections.
    
    Args:
        detections: List of (x, y, w, h, confidence) tuples
        overlap_threshold: IoU threshold for suppression
    
    Returns:
        filtered_detections: List of filtered detections
    """
    if len(detections) == 0:
        return []
    
    # Convert to numpy array for easier manipulation
    boxes = np.array([[d[0], d[1], d[0] + d[2], d[1] + d[3]] for d in detections])
    scores = np.array([d[4] for d in detections])
    
    # Sort by confidence (descending)
    indices = np.argsort(scores)[::-1]
    
    keep = []
    while len(indices) > 0:
        # Keep the box with highest confidence
        current = indices[0]
        keep.append(current)
        
        if len(indices) == 1:
            break
        
        # Calculate IoU with remaining boxes
        current_box = boxes[current]
        other_boxes = boxes[indices[1:]]
        
        # Calculate intersection
        x1 = np.maximum(current_box[0], other_boxes[:, 0])
        y1 = np.maximum(current_box[1], other_boxes[:, 1])
        x2 = np.minimum(current_box[2], other_boxes[:, 2])
        y2 = np.minimum(current_box[3], other_boxes[:, 3])
        
        intersection = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
        
        # Calculate union
        current_area = (current_box[2] - current_box[0]) * (current_box[3] - current_box[1])
        other_areas = (other_boxes[:, 2] - other_boxes[:, 0]) * (other_boxes[:, 3] - other_boxes[:, 1])
        union = current_area + other_areas - intersection
        
        # Calculate IoU
        iou = intersection / union
        
        # Remove boxes with high overlap
        indices = indices[1:][iou < overlap_threshold]
    
    return [detections[i] for i in keep]


def draw_detections(image, detections, color=(0, 255, 0), thickness=2):
    """
    Draw bounding boxes on image.
    
    Args:
        image: Input image (can be color or grayscale)
        detections: List of (x, y, w, h, confidence) tuples
        color: Bounding box color (BGR format)
        thickness: Line thickness
    
    Returns:
        image_with_boxes: Image with drawn bounding boxes
    """
    # Convert to color if grayscale
    if len(image.shape) == 2:
        image_with_boxes = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    else:
        image_with_boxes = image.copy()
    
    for x, y, w, h, confidence in detections:
        # Draw bounding box
        cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), color, thickness)
        
        # Draw confidence score
        label = f"Stop: {confidence:.2f}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image_with_boxes, (x, y - label_size[1] - 5), 
                     (x + label_size[0], y), color, -1)
        cv2.putText(image_with_boxes, label, (x, y - 5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    return image_with_boxes


In [ ]:
# Test on a few images from the stop signs folder
test_images_path = path_stop
test_files = [f for f in os.listdir(test_images_path) 
              if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))][:3]  # Test on first 3 images

fig, axes = plt.subplots(len(test_files), 2, figsize=(15, 5 * len(test_files)))

if len(test_files) == 1:
    axes = axes.reshape(1, -1)

for idx, filename in enumerate(test_files):
    img_path = os.path.join(test_images_path, filename)
    image = cv2.imread(img_path)
    
    if image is None:
        continue
    
    # Convert to grayscale for detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect stop signs
    print(f"\nDetecting in {filename}...")
    detections = detect_stop_signs(gray, model, window_size=(128, 128), 
                                   step_size=32, scales=[1.0, 0.75, 0.5, 1.25], 
                                   threshold=0.6)
    print(f"Found {len(detections)} detections before NMS")
    
    # Apply non-maximum suppression
    filtered_detections = non_max_suppression(detections, overlap_threshold=0.3)
    print(f"Found {len(filtered_detections)} detections after NMS")
    
    # Draw detections
    result_image = draw_detections(image, filtered_detections)
    
    # Display original and result
    axes[idx, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[idx, 0].set_title(f"Original: {filename}")
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    axes[idx, 1].set_title(f"Detections: {len(filtered_detections)} found")
    axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()


## Test on Non-Stop Sign Images (Should have no detections)


In [ ]:
# Test on a few images from the non-stop signs folder
test_images_path = path_non_stop
test_files = [f for f in os.listdir(test_images_path) 
              if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))][:3]  # Test on first 3 images

fig, axes = plt.subplots(len(test_files), 2, figsize=(15, 5 * len(test_files)))

if len(test_files) == 1:
    axes = axes.reshape(1, -1)

for idx, filename in enumerate(test_files):
    img_path = os.path.join(test_images_path, filename)
    image = cv2.imread(img_path)
    
    if image is None:
        continue
    
    # Convert to grayscale for detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect stop signs
    print(f"\nDetecting in {filename}...")
    detections = detect_stop_signs(gray, model, window_size=(128, 128), 
                                   step_size=32, scales=[1.0, 0.75, 0.5, 1.25], 
                                   threshold=0.6)
    print(f"Found {len(detections)} detections before NMS")
    
    # Apply non-maximum suppression
    filtered_detections = non_max_suppression(detections, overlap_threshold=0.3)
    print(f"Found {len(filtered_detections)} detections after NMS")
    
    # Draw detections
    result_image = draw_detections(image, filtered_detections)
    
    # Display original and result
    axes[idx, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[idx, 0].set_title(f"Original: {filename}")
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    axes[idx, 1].set_title(f"Detections: {len(filtered_detections)} found")
    axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()


## Notes on Parameters

- **window_size**: Size of the sliding window (should match training size: 128x128)
- **step_size**: How many pixels to move the window each step (smaller = more detections but slower)
- **scales**: Different scales to search at (1.0 = original size, 0.5 = half size, etc.)
- **threshold**: Confidence threshold for detections (higher = fewer false positives)
- **overlap_threshold**: IoU threshold for non-maximum suppression (higher = more overlapping boxes kept)

### Tips for Better Detection:
1. Adjust `threshold` to balance false positives vs false negatives
2. Use more scales for better detection at different sizes
3. Smaller `step_size` gives better coverage but is slower
4. Adjust `overlap_threshold` based on how many overlapping detections you expect
